In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
class ID3Dat:
    
    # Membuat fungsi untuk model klasifikasi dari data latih dan data uji yang telah ditetapkan
    def fit(self, input, output):
        data = input.copy()
        data[output.name] = output
        self.tree = self.decision_tree(data, data, input.columns, output.name)

    # Membuat fungsi predict
    def predict(self, input):
        #konversikan data input ke variabel samples
        samples = input.to_dict(orient='records')
        predictions = []

        #membuat prediksi untuk setiap sample
        for sample in samples:
            predictions.append(self.make_prediction(sample, self.tree, 1.0))

        return predictions

    # Mengaplikasikan rumus entropy
    def entropy(self, attribute_column):
        
        # menemukan nilai unik
        values, counts = np.unique(attribute_column, return_counts=True)

        # menghitung nilai entropy sesuai dengan nilai unik yang ditemukan
        entropy_list = []

        for i in range(len(values)):
            probability = counts[i]/np.sum(counts)
            entropy_list.append(-probability*np.log2(probability))

        # menjumlahkan setiap nilai entropy
        total_entropy = np.sum(entropy_list)

        return total_entropy

    def information_gain(self, data, feature_attribute_name, target_attribute_name):
        
        # menemukan total entropy
        total_entropy = self.entropy(data[target_attribute_name])

        # menemukan nilai unik dan menghitung masing-masing frekuensi untuk dipisahkan
        values, counts = np.unique(data[feature_attribute_name], return_counts=True)

        # menghitung total entropy setiap subset
        weighted_entropy_list = []

        for i in range(len(values)):
            subset_probability = counts[i]/np.sum(counts)
            subset_entropy = self.entropy(data.where(data[feature_attribute_name]==values[i]).dropna()[target_attribute_name])
            weighted_entropy_list.append(subset_probability*subset_entropy)

        total_weighted_entropy = np.sum(weighted_entropy_list)

        # menghitung information gain
        information_gain = total_entropy - total_weighted_entropy

        return information_gain

    # model decision tree dengan algoritma Id3
    def decision_tree(self, data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class=None):
        
        # jika data bersih, kembalikan nilai unik setiap subset
        unique_classes = np.unique(data[target_attribute_name])
        if len(unique_classes) <= 1:
            return unique_classes[0]
        # jika subset kosong, kembalikan nilai unik ke data originalnya
        elif len(data) == 0:
            majority_class_index = np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
            return np.unique(original_data[target_attribute_name])[majority_class_index]
        # jika data set tidak mengandung nilai untuk dilatih, kembalikan nilai ke parent node class
        elif len(feature_attribute_names) == 0:
            return parent_node_class
        # jika tidak terjadi seluruh syarat diatas
        else:
            majority_class_index = np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
            parent_node_class = unique_classes[majority_class_index]

            ig_values = [self.information_gain(data, feature, target_attribute_name) for feature in feature_attribute_names]
            best_feature_index = np.argmax(ig_values)
            best_feature = feature_attribute_names[best_feature_index]

            tree = {best_feature: {}}

            feature_attribute_names = [i for i in feature_attribute_names if i != best_feature]

            parent_attribute_values = np.unique(data[best_feature])
            for value in parent_attribute_values:
                sub_data = data.where(data[best_feature] == value).dropna()

                subtree = self.decision_tree(sub_data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class)

                tree[best_feature][value] = subtree

            return tree

    # membuat fungsi untukk prediksi
    def make_prediction(self, sample, tree, default=1):
        for attribute in list(sample.keys()):
          if attribute in list(tree.keys()):
            try:
                result = tree[attribute][sample[attribute]]
            except:
                return default

            result = tree[attribute][sample[attribute]]

            if isinstance(result, dict):
                return self.make_prediction(sample, result)
            else:
                return result

In [ ]:
df = pd.read_csv('D:\~~COLLAGE~~\SEMESTER 6\APM\P2\Wine.csv')

# organize data into input and output
X = df.drop(columns="Class")
y = df["Class"]

In [ ]:
X

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [ ]:
y

0      1
1      1
2      1
3      1
4      1
      ..
173    3
174    3
175    3
176    3
177    3
Name: Class, Length: 178, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

# initialize and fit model
model = ID3Dat()
model.fit(X_train, y_train)

# return accuracy score
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.6111111111111112